# **Classification**( essentials )

# **MNIST dataset**
Considerado el "Hello World" en ML es un conjunto de 70.000 pequeñas imágenes de dígitos escritas a mano por
estudiantes de secundaria y empleados de US Census Bureau

In [1]:
print('hello chapter 3')

hello chapter 3


it is out side of the commits.


In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

c:\Users\Manue!_PC\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:91: UserWarning: A network error occurred while downloading https://openml.org/api/v1/json/data/list/data_name/mnist_784/limit/2/data_version/1. Retrying...
  warn(


URLError: <urlopen error [Errno 11001] getaddrinfo failed>